<a href="https://colab.research.google.com/github/bethinaveen23/EXPLAINABLE-AI/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem:1

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame({
    "Temperature": [20, 22, 29, 41, 33],
    "IceCreamsSold": [200, 430, 190, 210, 250]
})

X, y = df[["Temperature"]], df["IceCreamsSold"]
model = LinearRegression().fit(X, y)

baseline = y.mean()
df["Prediction"] = model.predict(X)
df["SHAP"] = df["Prediction"] - baseline
df["Baseline+SHAP"] = baseline + df["SHAP"]
df["Diff"] = df["Prediction"] - y
df["Over/Under"] = df["Diff"].apply(lambda v: "Over" if v > 0 else "Under" if v < 0 else "Exact")

print(df[["Temperature", "IceCreamsSold", "Prediction", "SHAP", "Baseline+SHAP", "Over/Under"]])

for i, r in df.iterrows():
    e = "increase" if r.SHAP > 0 else "decrease" if r.SHAP < 0 else "no change"
    print(f"{r.Temperature}°C → {e} of {abs(r.SHAP):.2f} from baseline {baseline:.2f}")


   Temperature  IceCreamsSold  Prediction       SHAP  Baseline+SHAP Over/Under
0           20            200  296.034483  40.034483     296.034483       Over
1           22            430  287.137931  31.137931     287.137931      Under
2           29            190  256.000000   0.000000     256.000000       Over
3           41            210  202.620690 -53.379310     202.620690      Under
4           33            250  238.206897 -17.793103     238.206897      Under
20°C → increase of 40.03 from baseline 256.00
22°C → increase of 31.14 from baseline 256.00
29°C → no change of 0.00 from baseline 256.00
41°C → decrease of 53.38 from baseline 256.00
33°C → decrease of 17.79 from baseline 256.00


Problem:2

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame({
    "Emails": [200, 250, 220, 180, 60],
    "Social": [15, 20, 28, 14, 22],
    "Enroll": [320, 480, 400, 350, 280]
})

X, y = df[["Emails", "Social"]], df["Enroll"]
m = LinearRegression().fit(X, y)

b0, b1, b2 = m.intercept_, *m.coef_
base = y.mean()
mx1, mx2 = df["Emails"].mean(), df["Social"].mean()

df["Pred"] = m.predict(X)
df["SHAP_E"] = b1 * (df["Emails"] - mx1)
df["SHAP_S"] = b2 * (df["Social"] - mx2)
df["Base+SHAP"] = base + df["SHAP_E"] + df["SHAP_S"]
df["Err"] = df["Pred"] - y
df["Over/Under"] = df["Err"].apply(lambda e: "Over" if e > 0 else "Under" if e < 0 else "Exact")

print(df[["Emails", "Social", "Enroll", "Pred", "SHAP_E", "SHAP_S", "Base+SHAP", "Over/Under"]])

for i, r in df.iterrows():
    print(f"Row{i+1}: Emails {r.SHAP_E:.2f}, Social {r.SHAP_S:.2f} → Pred {r.Pred:.2f}, Act {r.Enroll}, {r['Over/Under']}")


   Emails  Social  Enroll        Pred      SHAP_E     SHAP_S   Base+SHAP  \
0     200      15     320  365.435279   15.903742 -16.468462  365.435279   
1     250      20     480  426.766988   60.080802   0.686186  426.766988   
2     220      28     400  427.708189   33.574566  28.133623  427.708189   
3     180      14     350  344.333526   -1.767082 -19.899392  344.333526   
4      60      22     280  265.756019 -107.792026   7.548045  265.756019   

  Over/Under  
0       Over  
1      Under  
2       Over  
3      Under  
4      Under  
Row1: Emails 15.90, Social -16.47 → Pred 365.44, Act 320, Over
Row2: Emails 60.08, Social 0.69 → Pred 426.77, Act 480, Under
Row3: Emails 33.57, Social 28.13 → Pred 427.71, Act 400, Over
Row4: Emails -1.77, Social -19.90 → Pred 344.33, Act 350, Under
Row5: Emails -107.79, Social 7.55 → Pred 265.76, Act 280, Under


Problem:3

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression().fit(X_train, y_train)
baseline = y_train.mean()
means = X_train.mean()

preds = model.predict(X_test)
shap_vals = (X_test - means) * model.coef_
shap_sum = shap_vals.sum(axis=1)

df = X_test.copy()
df["Actual"] = y_test
df["Pred"] = preds
df["Baseline+SHAP"] = baseline + shap_sum
df["Over/Under"] = df["Pred"] - df["Actual"]

print("Baseline:", baseline)
print("Coefficients:", dict(zip(X.columns, model.coef_)))
print(df.head())

for i, row in df.head(3).iterrows():
    print(f"\nPatient {i+1}: Pred={row.Pred:.1f}, Actual={row.Actual}, Error={row['Over/Under']:.1f}")
    for f in X.columns:
        val = shap_vals.loc[i, f]
        print(f"  {f}: {val:+.2f}")

Baseline: 153.73654390934846
Coefficients: {'age': np.float64(37.904021350074835), 'sex': np.float64(-241.96436231273972), 'bmi': np.float64(542.4287585162907), 'bp': np.float64(347.70384391385596), 's1': np.float64(-931.4888458835159), 's2': np.float64(518.0622769833373), 's3': np.float64(163.41998299131063), 's4': np.float64(275.3179015786486), 's5': np.float64(736.1988589046838), 's6': np.float64(48.67065743196508)}
          age       sex       bmi        bp        s1        s2        s3  \
287  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
211  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
72   0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
321  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
73   0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

           s4        s5        s6  Actual        Pred  Baseline+SHAP  \
287  0.034309  0.032432 -0.005220   219.0 

Problem:4

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import shap

df = pd.read_csv('/content/Experience_Salary.csv')
X = df.drop(columns=['Salary'])
y = df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression().fit(X_train, y_train)

baseline = y_train.mean()
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

preds = model.predict(X_test)
verify = baseline + shap_values.values.sum(axis=1)

for i in range(len(X_test)):
    print(f"Record {i+1}")
    print(f"Predicted: {preds[i]:.2f}, Actual: {y_test.iloc[i]}, Baseline: {baseline:.2f}")
    for f, val in zip(X.columns, shap_values.values[i]):
        print(f"{f}: {val:+.2f}")
    print(f"Sum SHAP + Baseline: {verify[i]:.2f}")
    print()

Record 1
Predicted: 115790.21, Actual: 112635, Baseline: 74207.62
YearsExperience: +41582.59
Sum SHAP + Baseline: 115790.21

Record 2
Predicted: 71498.28, Actual: 67938, Baseline: 74207.62
YearsExperience: -2709.35
Sum SHAP + Baseline: 71498.28

Record 3
Predicted: 102596.87, Actual: 113812, Baseline: 74207.62
YearsExperience: +28389.24
Sum SHAP + Baseline: 102596.87

Record 4
Predicted: 75267.80, Actual: 83088, Baseline: 74207.62
YearsExperience: +1060.18
Sum SHAP + Baseline: 75267.80

Record 5
Predicted: 55477.79, Actual: 64445, Baseline: 74207.62
YearsExperience: -18729.83
Sum SHAP + Baseline: 55477.79

Record 6
Predicted: 60189.70, Actual: 57189, Baseline: 74207.62
YearsExperience: -14017.93
Sum SHAP + Baseline: 60189.70

